In [22]:
from functools import partial
import operator
from typing import Annotated, Sequence, TypedDict, Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, BaseMessage
from pydantic import BaseModel
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
import keyring
import os
from display_graph import display_graph

# API KEY
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
TAVILY_API_KEY = keyring.get_password('tavily', 'key_for_mac')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

# Tavily tool
tavily_tool = TavilySearchResults(max_results=5)

# Code execution tool
python_repl_tool = PythonREPLTool()

# Set up LangSmith observability
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = keyring.get_password('langsmith', 'learning_agent')
os.environ['LANGCHAIN_PROJECT'] = "pr-stupendous-hood-8"

In [23]:
# Define RouteResponse for Customer Service Supervisor
class RouteResponseCS(BaseModel):
    next: Literal["Query_Agent", "Resolution_Agent", "Escalation_Agent", "FINISH"]

# Setup for Customer Service Supervisor
members_cs = ["Query_Agent", "Resolution_Agent", "Escalation_Agent"]
system_prompt_cs = f"You are a Customer Service Supervisor managing agents: {', '.join(members_cs)}."

# Create prompt template for the supervisor with correctly formatted options
prompt_cs = ChatPromptTemplate.from_messages([
    ("system", system_prompt_cs),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Choose the next agent to act from {options}."),
]).partial(options=str(members_cs))

In [24]:
# Define LLM and Supervisor function
llm = ChatOpenAI(model='gpt-4o-mini')

def supervisor_agent_cs(state):
    supervisor_chain_cs = prompt_cs | llm.with_structured_output(RouteResponseCS)
    return supervisor_chain_cs.invoke(state)

# Agent node function to handle message flow to each agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [HumanMessage(content=result["messages"][-1].content, name=name)]
    }

In [25]:
# Define agents for Customer Service tasks with realistic tools
query_agent = create_react_agent(llm, tools=[tavily_tool])
resolution_agent = create_react_agent(llm, tools=[python_repl_tool])
escalation_agent = create_react_agent(llm, tools=[python_repl_tool])

In [26]:
# Create nodes for each agent with valid names
query_node = partial(agent_node, agent=query_agent, name="Query_Agent")
resolution_node = partial(agent_node, agent=resolution_agent, name="Resolution_Agent")
escalation_node = partial(agent_node, agent=escalation_agent, name="Escalation_Agent")

In [27]:
# Define Customer Service graph state and workflow
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str
    
# Initialize StateGraph and add nodes
workflow_cs = StateGraph(AgentState)
workflow_cs.add_node("Query_Agent", query_node)
workflow_cs.add_node("Resolution_Agent", resolution_agent)
workflow_cs.add_node("Escalation_Agent", escalation_node)
workflow_cs.add_node("Supervisor", supervisor_agent_cs)

# Define edges for agents to return to the supervisor
for member in members_cs:
    workflow_cs.add_edge(member, "Supervisor")
    
# Define conditional map for routing
conditional_map_cs = {k: k for k in members_cs}
conditional_map_cs["FINISH"] = END
workflow_cs.add_conditional_edges("Supervisor", lambda x: x["next"], conditional_map_cs)
workflow_cs.add_edge(START, "Supervisor")

# Compile the grpah
graph_cs = workflow_cs.compile()

# Visualize the grpah
display_graph(graph_cs)

/Users/woojin/Documents/github/learning/llm/langgraph/langgraph_blueprint/ch13/graphs/graph_36349.png


In [32]:
# Example input for testing
inputs_cs = {"messages": [HumanMessage(content="Help me reset my password.")]}

# Run the graph
for output in graph_cs.stream(inputs_cs, {"recursion_limit":10}):
    if "__end__" not in output:
        print(output)

{'Supervisor': {'next': 'Resolution_Agent'}}
{'Resolution_Agent': {'messages': [HumanMessage(content='Help me reset my password.', additional_kwargs={}, response_metadata={}, id='59535a53-5292-43f0-8f21-62fff4e6ea19'), AIMessage(content='I can guide you through the steps to reset your password. Could you please specify which service or platform you need help with (e.g., email, social media, online banking, etc.)?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 84, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-0f98d600-92f1-4401-a4e9-1628faff229a-0', usage_metadata={'input_tokens': 84, 'output_tokens': 41, 'total_

GraphRecursionError: Recursion limit of 10 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT